In [2]:
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
from glob import glob
from tqdm import tqdm
import numpy as np

IN_PATH  = "./dataset/train/"
OUT_PATH = "./dataset/train_resized_640/"
RESIZE_DIM = (640, 640)

In [2]:
img_paths = glob(IN_PATH + "*.jpg")
print(len(img_paths))

30083


### resize `640x640`

In [74]:
for img_path in tqdm(img_paths):
    
    # read
    img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
    
    # resize image
    resized = cv2.resize(img, RESIZE_DIM, interpolation=cv2.INTER_AREA)

    # save image
    resize_name = OUT_PATH + img_path.split("/")[-1]
    #print(resize_name[:-3] + "png")
    status = cv2.imwrite(resize_name[:-3] + "png", resized)
    
    if status is False:
        print(img_path)

100%|██████████| 30083/30083 [15:32<00:00, 32.25it/s]


### resize `1024x1024`

In [2]:
OUT_PATH_512 = "./dataset/train_resized_1024/"
RESIZE_DIM = (1024, 1024)

img_paths = glob(IN_PATH + "*.jpg")
print(len(img_paths))


for img_path in tqdm(img_paths):
    
    # read
    img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
    
    # resize image
    resized = cv2.resize(img, RESIZE_DIM, interpolation=cv2.INTER_AREA)

    # save image
    resize_name = OUT_PATH_512 + img_path.split("/")[-1]
    status = cv2.imwrite(resize_name[:-3] + "png", resized)
    
    if status is False:
        print(img_path)

  0%|          | 3/30083 [00:00<19:15, 26.04it/s]

30083


100%|██████████| 30083/30083 [18:59<00:00, 26.40it/s]


### resize `512x512`

In [76]:
OUT_PATH_512 = "./dataset/train_resized_512/"
RESIZE_DIM = (512, 512)

img_paths = glob(IN_PATH + "*.jpg")
print(len(img_paths))


for img_path in tqdm(img_paths):
    
    # read
    img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
    
    # resize image
    resized = cv2.resize(img, RESIZE_DIM, interpolation=cv2.INTER_AREA)

    # save image
    resize_name = OUT_PATH_512 + img_path.split("/")[-1]
    status = cv2.imwrite(resize_name[:-3] + "png", resized)
    
    if status is False:
        print(img_path)

  0%|          | 5/30083 [00:00<12:15, 40.87it/s]

30083


100%|██████████| 30083/30083 [15:01<00:00, 33.44it/s]

KeyboardInterrupt: 

## Resize 1024 as `.npy`

In [3]:
import pandas as pd
csv_file = pd.read_csv("./dataset/train.csv")

In [ ]:

def save_as_numpy(input_dir, output_path, meta_file, size=(1024, 1024), ext="png"):
    
    arr = np.zeros((len(meta_file), *size), dtype="uint8")
    
    for idx, img_id in enumerate(tqdm(meta_file["StudyInstanceUID"].values)):
        img_path = input_dir + f"{img_id}.{ext}"
        img = cv2.imread(str(img_path), cv2.IMREAD_GRAYSCALE)
        arr[idx] = img
        
    np.save(output_path, arr)
    return arr


PATH_1024 = "./dataset/train_resized_1024/"
train_csv = pd.read_csv("./dataset/train.csv")
train_arr_320 = save_as_numpy(PATH_1024, "./dataset/train_1024x1024.npy", train_csv)
train_arr_320.shape


In [21]:
import numpy as np 
import pandas as pd 
import os
from sklearn.cluster import KMeans
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from collections import Counter, defaultdict
from sklearn.utils import check_random_state

train = pd.read_csv('./dataset/train.csv')
train.head()

,StudyInstanceUID,ETT - Abnormal,ETT - Borderline,ETT - Normal,NGT - Abnormal,NGT - Borderline,NGT - Incompletely Imaged,NGT - Normal,CVC - Abnormal,CVC - Borderline,CVC - Normal,Swan Ganz Catheter Present,PatientID
0,1.2.826.0.1.3680043.8.498.26697628953273228189...,0,0,0,0,0,0,1,0,0,0,0,ec89415d1
1,1.2.826.0.1.3680043.8.498.46302891597398758759...,0,0,1,0,0,1,0,0,0,1,0,bf4c6da3c
2,1.2.826.0.1.3680043.8.498.23819260719748494858...,0,0,0,0,0,0,0,0,1,0,0,3fc1c97e5
3,1.2.826.0.1.3680043.8.498.68286643202323212801...,0,0,0,0,0,0,0,1,0,0,0,c31019814
4,1.2.826.0.1.3680043.8.498.10050203009225938259...,0,0,0,0,0,0,0,0,0,1,0,207685cd1


In [22]:
class RepeatedStratifiedGroupKFold():

    def __init__(self, n_splits=5, n_repeats=1, random_state=None):
        self.n_splits = n_splits
        self.n_repeats = n_repeats
        self.random_state = random_state
        
    def split(self, X, y=None, groups=None):
        k = self.n_splits
        def eval_y_counts_per_fold(y_counts, fold):
            y_counts_per_fold[fold] += y_counts
            std_per_label = []
            for label in range(labels_num):
                label_std = np.std(
                    [y_counts_per_fold[i][label] / y_distr[label] for i in range(k)]
                )
                std_per_label.append(label_std)
            y_counts_per_fold[fold] -= y_counts
            return np.mean(std_per_label)
            
        rnd = check_random_state(self.random_state)
        for repeat in range(self.n_repeats):
            labels_num = np.max(y) + 1
            y_counts_per_group = defaultdict(lambda: np.zeros(labels_num))
            y_distr = Counter()
            for label, g in zip(y, groups):
                y_counts_per_group[g][label] += 1
                y_distr[label] += 1

            y_counts_per_fold = defaultdict(lambda: np.zeros(labels_num))
            groups_per_fold = defaultdict(set)
        
            groups_and_y_counts = list(y_counts_per_group.items())
            rnd.shuffle(groups_and_y_counts)

            for g, y_counts in sorted(groups_and_y_counts, key=lambda x: -np.std(x[1])):
                best_fold = None
                min_eval = None
                for i in range(k):
                    fold_eval = eval_y_counts_per_fold(y_counts, i)
                    if min_eval is None or fold_eval < min_eval:
                        min_eval = fold_eval
                        best_fold = i
                y_counts_per_fold[best_fold] += y_counts
                groups_per_fold[best_fold].add(g)
            
            all_groups = set(groups)
            for i in range(k):
                train_groups = all_groups - groups_per_fold[i]
                test_groups = groups_per_fold[i]

                train_indices = [i for i, g in enumerate(groups) if g in train_groups]
                test_indices = [i for i, g in enumerate(groups) if g in test_groups]

                yield train_indices, test_indices

In [23]:
# let's first concat all the labels 
# e.g 00000000010   
target_cols = train.drop(['StudyInstanceUID', 'PatientID'],axis=1).columns.values.tolist()
targets = train[target_cols].astype(str)
# create a new col to store the label
train['combined_tar'] = ''
for i in tqdm(range(targets.shape[1])):
    train['combined_tar'] += targets.iloc[:,i]
# take a look at it
train.combined_tar.value_counts()

100%|██████████| 11/11 [00:00<00:00, 468.46it/s]


00000000010    12763
00000000100     4038
00000001000     1678
00100010010     1428
00000000110      973
               ...  
00100000101        1
00110011000        1
01000001100        1
00100110011        1
00000100101        1
Name: combined_tar, Length: 211, dtype: int64

In [24]:
train['combined_tar'] = LabelEncoder().fit_transform(train['combined_tar'])

In [27]:
train['fold'] = -1
rskf = RepeatedStratifiedGroupKFold(n_splits=5, random_state=42)
for i, (train_idx, valid_idx) in tqdm(enumerate(rskf.split(train, train.combined_tar, train.PatientID))): #(df, targets, group)
    train.loc[valid_idx, 'fold'] = int(i)

5it [01:00, 12.00s/it]
